<a href="https://colab.research.google.com/github/liamford1/spam-detection-rnn/blob/main/SpamDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from google.colab import files
from sklearn.model_selection import train_test_split
uploaded = files.upload()


Saving spam.csv to spam (4).csv


In [89]:
data = pd.read_csv('spam.csv')
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [90]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=11, stratify=data['Category'])
print(data_train.shape)
print(data_test.shape)

(4457, 2)
(1115, 2)


In [91]:
X_train = data_train['Message']
y_train = data_train['Category']
X_test = data_test['Message']
y_test = data_test['Category']

In [92]:
y_train = y_train.map({'spam': 1, 'ham': 0})
y_test = y_test.map({'spam': 1, 'ham': 0})

In [86]:
X_train = data_train['message'].values
X_test = data_test['message'].values

In [93]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
words = tokenizer.index_word


In [94]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=20, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=20, padding='post')
X_train_pad[:5]
X_train_pad.shape

(4457, 20)

In [96]:
model = Sequential()
model.add(Embedding(input_dim=7983, output_dim=20))
model.add(LSTM(400))
model.add(Dense(1, activation='sigmoid'))

In [98]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [99]:
history = model.fit(
    X_train_pad, y_train,
    epochs=10,
    batch_size=64,
    validation_data=(X_test_pad, y_test)
)

Epoch 1/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 17s 204ms/step - accuracy: 0.8881 - loss: 0.3517 - val_accuracy: 0.9767 - val_loss: 0.0826
Epoch 2/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 20s 197ms/step - accuracy: 0.9839 - loss: 0.0625 - val_accuracy: 0.9848 - val_loss: 0.0667
Epoch 3/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 20s 196ms/step - accuracy: 0.9905 - loss: 0.0373 - val_accuracy: 0.9821 - val_loss: 0.0602
Epoch 4/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 21s 198ms/step - accuracy: 0.9968 - loss: 0.0160 - val_accuracy: 0.9848 - val_loss: 0.0727
Epoch 5/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 21s 204ms/step - accuracy: 0.9972 - loss: 0.0149 - val_accuracy: 0.9794 - val_loss: 0.0923
Epoch 6/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 20s 203ms/step - accuracy: 0.9980 - loss: 0.0155 - val_accuracy: 0.9794 - val_loss: 0.1100
Epoch 7/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 20s 198ms/step - accuracy: 0.9995 - loss: 0.0044 - val_accuracy: 0.9794 - val_loss: 0.0992
Epoch 8/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 14s 198ms/step - accuracy: 0.9993 - loss: 0.0065 - val_accu

In [100]:
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.9800 - loss: 0.1083
Loss: 0.11590077728033066
Accuracy: 0.9775784611701965


In [101]:
model.save('spam_detection.keras')

/content
['.config', 'spam.csv', 'spam (1).csv', 'spam_detection_rnn.h5', 'spam_detection_model.keras', 'sample_data']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>